# Test Project
## Prediction-based Trading and Object-Oriented Programming

### Import Necessary Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
# used to understand model performance and output (in an earlier version of the notebook)
#from sklearn.metrics import accuracy_score, confusion_matrix 
#from sklearn.metrics import classification_report

# Set Seaborn style and format for matplotlib figures
sns.set()
%config InlineBackend.figure_format = 'svg'

### Set Global Variables
**Sets  global variables, such as the URL of the data source, the symbol of the security to be analyzed, the number of lags to be used in the model, the initial amount of money to be used for trading, the transaction cost, and a verbose flag for logging/debugging.**

In [2]:
url = 'http://hilpisch.com/ref_eikon_eod_data.csv'
symbol = 'AAPL.O'
lags = 5
amount = 10000
ptc = 0.01
verbose = False # used for debugging and understanding the code/model outputs

### Scikit Models
**Defining a dictionary of machine learning models from Scikit-Learn to be used for prediction. The models include Gaussian Naive Bayes, Logistic Regression, Decision Tree Classifier, Support Vector Machine, and Multi-Layer Perceptron.**

In [3]:
models = {'gauss': GaussianNB(),
          'logreg': LogisticRegression(C=1, solver='lbfgs', max_iter=500),
          'dtc': DecisionTreeClassifier(max_depth=7),
          'svm': SVC(C=1, gamma='auto', kernel='linear'),
          'mlp': MLPClassifier(hidden_layer_sizes=[64], shuffle=False,
                              max_iter=2000)}

### FinancialData Class
**This class is responsible for data preparation and model training. It reads the data from the provided URL, creates various features, splits the data into training and testing sets, normalizes the data, adds lagged features, trains the model, and applies the model to the test data.**

In [4]:
class FinancialData:
    def __init__(self, url: str, symbol: str, model):
        self.model = model
        self.symbol = symbol
        # Data preparation: data pulled as "raw" from source
        self.raw = pd.read_csv(url, index_col=0, parse_dates=True)
        self.raw.dropna(inplace=True)
        # Data preparation: selected symbol selected and 11 features created
        self.data = pd.DataFrame(self.raw[symbol].iloc[:1000])
        self.data['r'] = np.log(self.data[symbol] / self.data[symbol].shift(1))
        self.data.dropna(inplace=True)
        self.data['d'] = np.where(self.data['r'] > 0, 1, 0)
        self.bins = [-0.01, -0.005, 0.005, 0.01]
        self.data['d_'] = np.digitize(self.data['r'], self.bins)
        self.data['SMA1'] = self.data[symbol].rolling(20).mean()
        self.data['SMA2'] = self.data[symbol].rolling(60).mean()
        self.data['SMA_'] = self.data['SMA1'] - self.data['SMA2']
        self.data['EWMA1'] = self.data[symbol].ewm(halflife=20).mean()
        self.data['EWMA2'] = self.data[symbol].ewm(halflife=60).mean()
        self.data['EWMA_'] = self.data['EWMA1'] - self.data['EWMA2']
        self.data['V1'] = self.data['r'].rolling(20).std()
        self.data['V2'] = self.data['r'].rolling(60).std()
        self.data.dropna(inplace=True)
        self.lags = 5
    
    # Data split into training and testing data
    def split_data(self):
        self.train, self.test = train_test_split(self.data, test_size=0.3, random_state=42, shuffle=False)
    
    # Data normalization on 11 features (only on training data) required for model training
    def normalize_data(self):
        self.scaler = StandardScaler()
        self.cols = ['SMA1', 'SMA2', 'SMA_', 'EWMA1', 'EWMA2', 'EWMA_', 'V1', 'V2']
        self.train[self.cols] = self.scaler.fit_transform(self.train[self.cols])
    
    # Lagged features created for training (total of 55 features)
    def add_lags(self, lags):
        self.cols.extend(['r', 'd', 'd_'])
        self.cols_ = list()
        for self.col in self.cols:
            for self.lag in range(1, lags + 1):
                self.col_ = self.col + f'_lag_{self.lag}'
                self.train[self.col_] = self.train[self.col].shift(self.lag)
                self.test[self.col_] = self.test[self.col].shift(self.lag)
                self.cols_.append(self.col_)
        self.train.dropna(inplace=True)
        self.test.dropna(inplace=True)
    
    # Model training
    def train_model(self):
        self.model.fit(self.train[self.cols_], self.train['d'])
    
    # Model applied to test data used by the BacktestingBase class function run_strategy method
    def apply_model(self):
        self.train['prediction'] = self.model.predict(self.train[self.cols_])
        self.test['prediction'] = self.model.predict(self.test[self.cols_])
    
    # Helper method to look at in-sample and out-of-sample model performance
    def evaluate_model(self, set_name):
        if set_name == "train":
            y_true = self.train['d']
            y_pred = self.train['prediction']
        elif set_name == "test":
            y_true = self.test['d']
            y_pred = self.test['prediction']
        else:
            print("Invalid set name. Choose between 'train' and 'test'.")
            return
        
        report = classification_report(y_true, y_pred, zero_division=1)
        print(f"Classification Report for {set_name} set: \n {report}")
    
    # Helpder method used during debugging to check whether predictions (1 or 0) are made
    def print_prediction_counts(self):
        counts = self.test['prediction'].value_counts()
        print(counts)
    
    # Helper method to visualize data
    def plot_data(self, cols=None):
        if cols is None:
            cols = self.symbol
        self.data[cols].plot(figsize=(10, 6), title=f'{self.symbol}')

### Backtesting Base Class Function
**This class inherits from the FinancialData class and is responsible for backtesting the trading strategy. It keeps track of the current balance, the number of units of the stock owned, and the number of trades made. It also has methods for placing buy and sell orders, closing out the position, and running the strategy.**

In [5]:
class BacktestingBase(FinancialData):
    def __init__(self, url, symbol, model, amount, ptc, verbose=False):
        super().__init__(url, symbol, model)
        self.initial_amount = amount
        self.current_balance = amount
        self.ptc = ptc
        self.units = 0
        self.trades = 0
        self.wealth_over_time = []
        self.verbose = verbose
        self.split_data()
        self.normalize_data()
        self.add_lags(self.lags)
        self.train_model()
        self.apply_model()

    def get_date_price(self, bar):
        date = str(self.data.index[bar])[:10]
        price = self.data[self.symbol].iloc[bar]
        return date, price
    
    def print_balance(self, bar):
        date, price = self.get_date_price(bar)
        print(f'{date} | current balance = {self.current_balance:.2f}')
    
    def print_net_wealth(self, bar):
        date, price = self.get_date_price(bar)
        net_wealth = self.current_balance + self.units * price
        print(f'{date} | net wealth = {net_wealth:.2f}')
    
    def place_buy_order(self, bar, units=None, amount=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / (price * (1 + self.ptc)))
        self.units += units
        self.current_balance -= units * price * (1 + self.ptc)
        self.trades += 1
        if self.verbose:
            print(f'{date} | bought {units} units for {price:.2f}')
            self.print_balance(bar)
            self.print_net_wealth(bar)
    
    def place_sell_order(self, bar, units=None, amount=None):
        date, price = self.get_date_price(bar)
        if units is None:
            units = int(amount / (price * (1 + self.ptc)))
        self.units -= units
        self.current_balance += units * price * (1 - self.ptc)
        self.trades += 1
        if self.verbose:
            print(f'{date} | sold {units} units for {price:.2f}')
            self.print_balance(bar)
            self.print_net_wealth(bar)
    
    def close_out(self, bar):
        date, price = self.get_date_price(bar)
        print(92 * '=')
        print(f'{date} | *** CLOSING OUT POSITION ***')
        print(92 * '=')
        print(f'{date} | closing {self.units} units for {price:.2f}')
        self.current_balance += self.units * price
        self.units = 0
        self.trades += 1
        self.print_balance(bar)
        perf = (self.current_balance / self.initial_amount - 1) * 100
        print(f'{date} | performance [%] = {perf:.3f}')
        print(f'{date} | trades [#] = {self.trades}\n\n')
        
    def print_net_wealth(self, bar):
        date, price = self.get_date_price(bar)
        net_wealth = self.current_balance + self.units * price
        self.wealth_over_time.append(net_wealth)
        print(f'{date} | net wealth = {net_wealth:.2f}')
        
    def run_strategy(self):
        print(92 * '=')
        print(f'*** BACKTESTING STRATEGY ***')
        print(f'{self.symbol} | lags={self.lags} | model={model}')
        print(92 * '=')

        # Adjust the loop to only access the test data
        for test_bar in range(len(self.test)):
            # Adjust bar for accessing self.data
            bar = test_bar + len(self.train)
            trade = False
            prediction = self.test['prediction'].iloc[test_bar]

            if self.verbose:
                # Print the prediction and units state at each bar (used for debugging)
                print(f"Bar #{bar}, Date: {self.data.index[bar]}, Prediction: {prediction}, Units: {self.units}")  
            if self.units == 0 or self.units == -1:
                if prediction == 1:
                    if self.verbose:
                        # Print when a buy order is placed
                        print(f"Placing a buy order at bar #{bar}, Date: {self.data.index[bar]}")  
                    # Buy with half of the current balance
                    self.place_buy_order(bar, amount=self.current_balance/2)  
                    trade = True
            else:  # self.units == 0 or self.units == 1
                if prediction == 0:
                    if self.verbose:
                        # Print when a sell order is placed
                        print(f"Placing a sell order at bar #{bar}, Date: {self.data.index[bar]}")
                    self.place_sell_order(bar, units=self.units)  # sell all
                    trade = True
                    
            if trade and self.verbose:
                self.print_balance(bar)
                self.print_net_wealth(bar)
                print(92 * '-')
                
        self.close_out(bar)

In [6]:
# Dictionary to store the performance of each model
performance = {}

# Loop over the models
for model_name, model in models.items():
    print(f"Running strategy with {model_name} model...")
    bt = BacktestingBase(url, symbol, model, amount, ptc, verbose)
    bt.run_strategy()
    performance[model_name] = bt.current_balance  # Store the final balance as performance metric

# Compare performance
for model_name, final_balance in performance.items():
    print(f"Model {model_name} final balance: {final_balance:.2f}")

Running strategy with gauss model...
*** BACKTESTING STRATEGY ***
AAPL.O | lags=5 | model=GaussianNB()
2013-12-06 | *** CLOSING OUT POSITION ***
2013-12-06 | closing 0 units for 80.00
2013-12-06 | current balance = 10000.00
2013-12-06 | performance [%] = 0.000
2013-12-06 | trades [#] = 1


Running strategy with logreg model...
*** BACKTESTING STRATEGY ***
AAPL.O | lags=5 | model=LogisticRegression(C=1, max_iter=500)
2013-12-06 | *** CLOSING OUT POSITION ***
2013-12-06 | closing 58 units for 80.00
2013-12-06 | current balance = 9647.98
2013-12-06 | performance [%] = -3.520
2013-12-06 | trades [#] = 2


Running strategy with dtc model...
*** BACKTESTING STRATEGY ***
AAPL.O | lags=5 | model=DecisionTreeClassifier(max_depth=7)
2013-12-06 | *** CLOSING OUT POSITION ***
2013-12-06 | closing 55 units for 80.00
2013-12-06 | current balance = 8567.58
2013-12-06 | performance [%] = -14.324
2013-12-06 | trades [#] = 24


Running strategy with svm model...
*** BACKTESTING STRATEGY ***
AAPL.O | lag

### General Areas for Potential Improvement
**Roadmap for Future Work**

1. **Error Handling**: The code could benefit from more robust error handling. For example, when reading the data from the URL, it would be helpful to handle potential exceptions that could occur if the URL is not accessible or the data is not in the expected format.

2. **Code Modularity**: The FinancialData class has a lot of included methods, from data loading to feature engineering to model training. It might be beneficial to break this down into smaller, more focused classes or functions - especially if we are going to modify and change the feature set.

3. **Feature Engineering**: The features used for the model are relatively simple. More work on optimizing and creating new features could potentially improve the model's performance.

4. **Model Evaluation**: The notebook currently only uses the final balance as a performance metric for the trading strategy. It would be helpful to include more detailed evaluation of the model's performance, such as precision, recall, F1 score, ROC AUC score, in the model-tuning phase of work. The _accuracy_score_ and _confusion_matrix_ were used in earlier notebook versions, but more model-tuning work would be required to properly utilize these tools. I would also like to do more work to better understand the derivative (and more advanced) classification performance metrics such as  precision, recall, and the F1 score - and how they would specifically apply in a binary classification problem such as ours (i.e., is the price going up or down?). Notwithstanding, I think the trading strategy has some fundamental flaws with respect to risk management. If we are getting two buy signals from the model in a row, should we be continuing to buy more of the security (adding risk), or should we just be holding onto our existing positions (as is currently modeled).

5. **Hyperparameter Tuning**: The models are used with default or arbitrarily chosen hyperparameters. Using techniques like grid search or random search to tune the hyperparameters could potentially improve the model's performance.

6. **Cross-Validation**: The model is trained on a single train-test split of the data. Using cross-validation would provide a more robust estimate of the model's performance.

7. **Trading Strategy/Algorithm Evaluation**: I may not have correctly implemented the long-short strategy as evidenced by the limited number of trades (and also performance statistics). For trading the APPL.O security, I expected the models to generally provide a "buy" signal but we are looking specifically at an earlier period in the super-cycle of a low rate environment, so we would need do more analysis on the time series data (i.e., cross-validation and also looking at the optimal daterange). Notwithstanding, if we are putting at risk 50% of our endowment, I expected a positive return (even though proportionate transaction costs are taken into account). I was also not able to replicate my vectorized backtesting results from Tutorial_05 even though I believe the long-short trading strategy implemented is basically the same(?).

### Trading Strategy Self-Review

1. **Risk management**: The algorithm currently doesn't have any risk management. It could be improved by incorporating a risk management strategy, such as setting a maximum loss limit or using stop-loss orders. The current strategy has a simple form of risk management by limiting the single-order size to 50% of the available investment capital.

2. **Order size**: The algorithm currently buys or sells based on 50% of the available capital. It could be improved by varying the order size based on the confidence of the prediction or the available balance.
    * We could change the order size based on the model's prediction confidence. We would  need to modify the model to output prediction probabilities instead of just the prediction class. Scikit-learn models have a _predict_proba_ method that can be used for this purpose. This method returns the probabilities for each class, which can be interpreted as the model's confidence in each prediction. Once you have the prediction probabilities, we can use them to determine the order size. For example, we could set the order size to be proportional to the prediction confidence.
    
    <br>

3. **Model performance**: The performance of the trading strategy is directly tied to the performance of the machine learning models. Improving the models, for example by tuning their parameters or using more advanced models, could improve the performance of the trading strategy. I would want to look at applying ARIMA-based and LSTM model on the current (and improved) set of features.

4. **Feature engineering**: The algorithm currently uses lagged returns as features. It could potentially be improved by using more sophisticated features.

5. **Market conditions**: The algorithm currently doesn't take into account market conditions. It could be improved by incorporating market indicators, such as volatility or market sentiment, into the trading decisions.

6. **Transaction Costs**: While the algorithm does take into account proportionate transaction costs, it might be worth exploring how sensitive the strategy is to these costs. For example, how would the strategy perform if transaction costs were higher or lower? This is particularly important if the strategy involves frequent trading.